In [1]:
import numpy as np

In [2]:
class TicTacToe:
    '''Handles tic-tac-toe board.
    
    Variables ===========
    
        board: array. Representation of the board. 0=empty, 1=first player, -1=second player.
        finished: boolean. 'True' if game is finished, 'False' otherwise.
        winner: int. '1' if first player won, '-1' if second player won, '0' otherwise.
        
    Methods =============
    
        available_actions() -> array
            Returns array of boolean values indicating valid moves. E.g. if all moves are valid, 
            then returns the 9-dimensional array [True, True, ..., True].
            
        play(move: int)
            Play a move/action. 'move' represents the flattened dimension, i.e. move = 3*y + x.
            Alternates between first and second players.
            
        find_winner() -> int
            Computes and returns the 'winner' variable.
    '''
    
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=np.int8)
        self.finished = False
        self.winner = 0
        self._curr_player = +1
        self._tot_moves = 0
        
    def available_actions(self):
        return (self.board == 0).flatten()
    
    def play(self, move):
        
        x, y = move//3, move%3
        
        assert not self.finished, "game has ended"
        assert self.board[x, y] == 0, "invalid move"
        
        self.board[x, y] = self._curr_player
        self._curr_player *= -1
        self._tot_moves += 1
        
        ## game ends if no more moves
        if self._tot_moves == 9:
            self.finished = True
        ## game ends when there is a winner
        self.winner = self.find_winner()
        if self.winner != 0:
            self.finished = True     
        
    def find_winner(self):
        
        s_col = np.sum(self.board, axis=0)
        if 3 in s_col:    return 1
        elif -3 in s_col: return -1
        
        s_row = np.sum(self.board, axis=1)
        if 3 in s_row:    return 1
        elif -3 in s_row: return -1
            
        s = self.board[0,0] + self.board[1,1] + self.board[2,2]
        if s == 3: return 1
        elif s == -3: return -1
        
        s = self.board[0,2] + self.board[1,1] + self.board[2,0]
        if s == 3: return 1
        elif s == -3: return -1
        
        return 0
    

    
class Player:
    '''Player of tic-tac-toe game.
    
    Parameters =========
    
        name: string. Name of player.
        piece: int. Either +1 for first player or -1 for second player.
        
    Methods ============
    
        play(game: TicTacToe) -> int
            Returns move/action. 'move' represents the flattened dimension, i.e. move = 3*x + y.
            
        learn(episode: list)
            Learn from episode.
    '''
    def __init__(self, name, piece):
        self.name = name
        self.piece = piece
        
    def play(self, game):
        pass
    
    def learn(self, episode):
        pass
        

In [6]:
class MCPlayer(Player):
    '''Q-function learned by MC-control.
    
    Parameters =========
    
        name.
        piece.
        eps: float. Probability for random action.
        discount: float. Discount factor.
            
    Variables ==========
    
        N: dict. Tracks number of state-action pairs.
        Q: dict. Estimator of Q-function.
    '''
    def __init__(self, name, piece, eps=.2, discount=.96):
        super().__init__(name, piece)
        self.eps = eps
        self.discount = discount
        self.Q = {}
        self.N = {}
    
    def play(self, game):
        
        available_actions = game.available_actions()
        x = np.random.random()
        
        ## play greedy
        if x > self.eps:
            s = (game.board * self.piece).tobytes()
            values = self.Q.get(s, np.zeros(9))
            best_value = np.amax(values[available_actions])
            actions = (values == best_value) * available_actions
        ## play random
        else:
            actions = available_actions
        
        return np.random.choice(np.where(actions)[0])
            
    def learn(self, episode):
    
        R = episode.pop()
        while episode:
            a = episode.pop()
            s = (episode.pop() * self.piece).tobytes()
            
            values = self.Q.get(s, np.zeros(9))
            self.Q[s] = values
            
            Ns = self.N.get(s, np.zeros(9))
            self.N[s] = Ns
            
            Ns[a] += 1.
            #values[a] += self.lr * (R - values[a])
            values[a] += (R - values[a]) / Ns[a]
            R *= self.discount
    

In [4]:
def play_game(p1, p2, p1_learn=False, p2_learn=False, verbose=False):
    
    game = TicTacToe()
    p1_ep = []
    p2_ep = []
    p_i = 0
    
    p1.piece = +1
    p2.piece = -1
    
    while not game.finished:
        
        ## which player's turn?
        if p_i == 0:
            p, ep, learn = p1, p1_ep, p1_learn
        else:
            p, ep, learn = p2, p2_ep, p2_learn

        ## player move
        move = p.play(game)
        if verbose: print("{0:s} plays move {1:d}".format(p.name, move))

        ## update episode
        if learn:
            ep.append(np.copy(game.board))
            ep.append(move)

        ## update game
        game.play(move)

        p_i = (p_i+1)%2
        
    if verbose: print(game.board)
    
    ## give winner reward 1, loser reward -1
    if game.winner == 1:
        if verbose: print("winner: {0:s}.".format(p1.name))
        p1_ep.append(1.)
        p2_ep.append(-1.)
    elif game.winner == -1:
        if verbose: print("winner: {0:s}.".format(p2.name))
        p1_ep.append(-1.)
        p2_ep.append(1.)
    else:
        if verbose: print("tie.")
        p1_ep.append(0.)
        p2_ep.append(0.)
    
    if p1_learn:
        p1.learn(p1_ep)
    if p2_learn:
        p2.learn(p2_ep)
        
    return game.winner

# Training
We train two agents against each other.

In [7]:
agent1 = MCPlayer('agent1', 0, eps=.2)
agent2 = MCPlayer('agent2', 0, eps=.2)

for i in range(100_000):
    play_game(agent1, agent2, p1_learn=True, p2_learn=True)
    play_game(agent2, agent1, p1_learn=True, p2_learn=True)

In [8]:
agent1.eps = 0
agent2.eps = 0

In [12]:
play_game(agent1, agent2, verbose=True)

agent1 plays move 4
agent2 plays move 2
agent1 plays move 5
agent2 plays move 3
agent1 plays move 0
agent2 plays move 8
agent1 plays move 7
agent2 plays move 1
agent1 plays move 6
[[ 1 -1 -1]
 [-1  1  1]
 [ 1  1 -1]]
tie.


0

We can also play against an agent.

In [25]:
game = TicTacToe()
#agent1.piece = -1 # agent goes second
agent1.piece = +1 # agent goes first

In [29]:
game.play(2)
game.board

array([[-1,  0, -1],
       [ 0,  1,  0],
       [ 1,  0,  0]], dtype=int8)

In [30]:
game.play(agent1.play(game))
game.board

array([[-1,  1, -1],
       [ 0,  1,  0],
       [ 1,  0,  0]], dtype=int8)